<a href="https://colab.research.google.com/github/myst9811/PEPE-CNN/blob/main/Pepe_the_Frog_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets

***IMPORTING DATASET AND NECESSARY LIBRARIES***

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import load_dataset,Image
dataset = load_dataset("vikhyatk/synthetic-pepe")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/505 [00:00<?, ?it/s]

In [4]:
import matplotlib.pyplot as plt
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 504
    })
})


In [5]:
from PIL import Image
import io


 ***IMAGE RESIZING FUNCTION***

In [31]:
def resize_image(example):
    """Resizes an image to a specified size.

    Parameters:
    - example: An example from the dataset.

    Returns:
    - The example with the resized image.
    """
    # Check if the image data is stored as a filepath (string) and load it, or directly handle it as an array
    if isinstance(example['image'], str):
        # If 'image' is a filepath, open it with PIL (this part might need to be adjusted based on your data)
        with open(example['image'], 'rb') as f:
            image = Image.open(f)
            image = image.convert('RGB')  # Ensure image is in RGB format
    else:
        # If 'image' is already an array, convert it to a PIL Image
        image_data = np.array(example['image'])
        image = Image.fromarray(image_data)

    # Resize the image
    image = image.resize((32, 32))

    # Convert the PIL Image back to an array
    example['image'] = np.array(image)

    return example

***SPLITTING THE DATASET INTO TRAINING, TESTING AND VALIDATION SETS***

In [41]:
# Assuming 'train' key exists and using the datasets' own method to split
split_dataset = dataset['train'].train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# Further split the training dataset into training and validation sets
split_train_dataset = train_dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_train_dataset['train']
val_data = split_train_dataset['test']

# Assuming resize_image is correctly defined to work with your dataset structure
# Apply the resize function using .map
train_data = train_data.map(resize_image, batched=False)
val_data = val_data.map(resize_image, batched=False)
test_dataset = test_dataset.map(resize_image, batched=False)


print(train_data.features)

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

{'image': Image(decode=True, id=None)}


In [27]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D, Input

***As the Data is non labled, We encode and decode the data***

In [42]:
# Define the encoder
input_img = Input(shape=(32, 32, 3))  # assuming 32x32 RGB images
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Define the decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Correctly defining the autoencoder model
autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Example of converting the datasets to NumPy arrays
# You might need to adjust this based on your specific dataset and preprocessing needs

train_images = np.array(train_data['image'])  # Replace 'image' with the actual image field name
train_images = train_images.astype('float32') / 255.  # Normalize the images

# If you have a separate validation dataset
val_images = np.array(val_data['image'])  # Replace 'image' with the actual image field name
val_images = val_images.astype('float32') / 255.  # Normalize the images

# Assuming the autoencoder is expecting input in the shape (32, 32, 3) and your data is already in this shape after preprocessing


autoencoder.fit(train_images, train_images, epochs=50, batch_size=256, shuffle=True,
                validation_data=(val_images, val_images))


Epoch 1/50


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

***IMPLEMENTING THE NEURAL NETWORK***

In [43]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D, Input


In [35]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Modified for binary classification
])


In [40]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [37]:
history = model.fit(train_data['image'], train_data['label'],
                    validation_data=(val_data['image'], val_data['label']),
                    epochs=10, batch_size=32)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset['image'], test_dataset['label'])
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


KeyError: "Column label not in the dataset. Current columns in the dataset: ['image']"